## Рекомендательные системы

#### Описание задачи

Небольшой интернет-магазин попросил вас добавить ранжирование товаров в блок "Смотрели ранее" - в нем теперь надо показывать не последние просмотренные пользователем товары, а те товары из просмотренных, которые он наиболее вероятно купит. Качество вашего решения будет оцениваться по количеству покупок в сравнении с прошлым решением в ходе А/В теста, т.к. по доходу от продаж статзначимость будет достигаться дольше из-за разброса цен. Таким образом, ничего заранее не зная про корреляцию оффлайновых и онлайновых метрик качества, в начале проекта вы можете лишь постараться оптимизировать recall@k и precision@k.  
Это задание посвящено построению простых бейзлайнов для этой задачи: ранжирование просмотренных товаров по частоте просмотров и по частоте покупок. Эти бейзлайны, с одной стороны, могут помочь вам грубо оценить возможный эффект от ранжирования товаров в блоке - например, чтобы вписать какие-то числа в коммерческое предложение заказчику, а с другой стороны, могут оказаться самым хорошим вариантом, если данных очень мало (недостаточно для обучения даже простых моделей).

Входные данные:
* Вам дается две выборки с пользовательскими сессиями - id-шниками просмотренных и id-шниками купленных товаров. Одна выборка будет использоваться для обучения (оценки популярностей товаров), а другая - для теста.
* В файлах записаны сессии по одной в каждой строке. Формат сессии: id просмотренных товаров через , затем идёт ; после чего следуют id купленных товаров (если такие имеются), разделённые запятой. Например, 1,2,3,4; или 1,2,3,4;5,6. Гарантируется, что среди id купленных товаров все различные.

Важно:
* Сессии, в которых пользователь ничего не купил, исключаем из оценки качества.
* Если товар не встречался в обучающей выборке, его популярность равна 0.
* Рекомендуем разные товары. И их число должно быть не больше, чем количество различных просмотренных пользователем товаров.
* Рекомендаций всегда не больше, чем минимум из двух чисел: количество просмотренных пользователем товаров и k в recall@k / precision@k.

In [2]:
import pandas as pd
import collections

In [8]:
trainData = pd.read_csv('coursera_sessions_train.txt', sep=';', header=None,
                          names=['viewedProducts', 'purchasedProducts'])
testData = pd.read_csv('coursera_sessions_test.txt', sep=';', header=None,
                          names=['viewedProducts', 'purchasedProducts'])

In [24]:
testData.dropna(inplace=True)

Построим частоты появления id в просмотренных и в купленных товарах.

In [9]:
viewFreq = collections.Counter((','.join(trainData.viewedProducts.tolist())).split(','))

In [11]:
trainData.dropna(inplace=True)
purchaseFreq = collections.Counter((','.join(trainData.purchasedProducts.tolist())).split(','))

Реализуем два алгоритма рекомендаций:
* сортировка просмотренных id по популярности (частота появления в просмотренных)
* сортировка просмотренных id по покупаемости (частота появления в покупках).

In [12]:
def sortedView(row, k = 0): 
    row = row.split(',')
    sortData = pd.DataFrame({'id': row,
                             'serial number': range(len(row)),
                             'frequence': [viewFreq[row[i]] for i in range(len(row))]})

    groupData = sortData.groupby(
        ['id','frequence']).agg({'serial number': min}).reset_index().sort_values(by=(['frequence','serial number']),
                                                                                  ascending=[0,1])
    
    if k <= 0:
        return(','.join(groupData.id))
    else:
        return(','.join(groupData.id[ : k]))


def sortedPurchase(row, k = 0):    
    row = row.split(',')
    sortData = pd.DataFrame({'id': row,
                             'serial number': range(len(row)),
                             'frequence': [purchaseFreq [row[i]] for i in range(len(row))]})
    groupData = sortData.groupby(
        ['id','frequence']).agg({'serial number': min}).reset_index().sort_values(by=(['frequence','serial number']), 
                                                                                  ascending=[0,1])
    if k <= 0:
        return(','.join(groupData.id))
    else:
        return(','.join(groupData.id[:k]))


In [13]:
trainData['sortedViews'] = trainData.viewedProducts.apply(lambda x: sortedView(x, k = 0))
trainData['recommendForView1'] = trainData.viewedProducts.apply(lambda x: sortedView(x, k = 1))

trainData['viewPrecision@1'] = trainData.apply(
    lambda x: len(set.intersection(set(x.purchasedProducts.split(',')),
                                   set(x.recommendForView1.split(',')))), axis=1)

trainData['viewRecall@1'] = trainData.apply(
    lambda x: len(set.intersection(set(x.purchasedProducts.split(',')), 
                                   set(x.recommendForView1.split(','))))/len(set(x.purchasedProducts.split(','))), 
                                                                                                                 axis=1)

In [14]:
trainData['recommendForView5'] = trainData.viewedProducts.apply(lambda x: sortedView(x, k = 5))

trainData['viewPrecision@5'] = trainData.apply(
    lambda x: len(set.intersection(set(x.purchasedProducts.split(',')),
                                   set(x.recommendForView5.split(','))))/5, axis=1)

trainData['viewRecall@5'] = trainData.apply(
    lambda x: len(set.intersection(set(x.purchasedProducts.split(',')), 
                                   set(x.recommendForView5.split(','))))/len(set(x.purchasedProducts.split(','))), 
                                                                                                                 axis=1)

In [15]:
trainData['sortedPurchase'] = trainData.viewedProducts.apply(lambda x: sortedPurchase(x, k = 0))
trainData['recommendForPurchase1'] = trainData.viewedProducts.apply(lambda x: sortedPurchase(x, k = 1))


trainData['purchasePrecision@1'] = trainData.apply(
    lambda x: len(set.intersection(set(x.purchasedProducts.split(',')), 
                                   set(x.recommendForPurchase1.split(',')))), axis=1)
trainData['purchaseRecall@1'] = trainData.apply(
    lambda x: len(set.intersection(set(x.purchasedProducts.split(',')), 
                                   set(x.recommendForPurchase1.split(','))))/(len(set(x.purchasedProducts.split(',')))), 
                                                                                                                 axis=1)

In [21]:
trainData['recommendForPurchase5'] = trainData.viewedProducts.apply(lambda x: sortedPurchase(x, k = 5))
trainData['purchasePrecision@5'] = trainData.apply(
    lambda x: len(set.intersection(set(x.purchasedProducts.split(',')), 
                                   set(x.recommendForPurchase5.split(','))))/5, axis=1)
trainData['purchaseRecall@5'] = trainData.apply(
    lambda x: len(set.intersection(set(x.purchasedProducts.split(',')), 
                                   set(x.recommendForPurchase5.split(','))))/len(set(x.purchasedProducts.split(','))), 
                                                                                                                 axis=1)

In [22]:
trainData.head()

,viewedProducts,purchasedProducts,sortedViews,recommendForView1,viewPrecision@1,viewRecall@1,recommendForView5,viewPrecision@5,viewRecall@5,sortedPurchase,recommendForPurchase1,purchasePrecision@1,purchaseRecall@1,recommendForPurchase5,purchasePrecision@5,purchaseRecall@5
7,"59,60,61,62,60,63,64,65,66,61,67,68,67","67,60,63","63,64,60,61,65,66,67,68,59,62",63,1,0.333333,"63,64,60,61,65",0.4,0.666667,"60,63,67,59,61,62,64,65,66,68",60,1,0.333333,"60,63,67,59,61",0.6,1.0
10,"84,85,86,87,88,89,84,90,91,92,93,86",86,"85,93,89,90,84,92,86,87,91,88",85,0,0.000000,"85,93,89,90,84",0.0,0.000000,"86,85,93,84,87,88,89,90,91,92",86,1,1.000000,"86,85,93,84,87",0.2,1.0
19,"138,198,199,127",199,"127,138,198,199",127,0,0.000000,"127,138,198,199",0.2,1.000000,"138,199,127,198",138,0,0.000000,"138,199,127,198",0.2,1.0
30,"303,304,305,306,307,308,309,310,311,312",303,"303,306,304,307,309,310,305,308,311,312",303,1,1.000000,"303,306,304,307,309",0.2,1.000000,"303,304,305,306,307,308,309,310,311,312",303,1,1.000000,"303,304,305,306,307",0.2,1.0
33,"352,353,352",352,"352,353",352,1,1.000000,"352,353",0.2,1.000000,"352,353",352,1,1.000000,"352,353",0.2,1.0


In [25]:
testData['sortedViews'] = testData.viewedProducts.apply(lambda x: sortedView(x, k = 0))
testData['recommendForView1'] = testData.viewedProducts.apply(lambda x: sortedView(x, k = 1))

testData['viewPrecision@1'] = testData.apply(
    lambda x: len(set.intersection(set(x.purchasedProducts.split(',')),
                                   set(x.recommendForView1.split(',')))), axis=1)

testData['viewRecall@1'] = testData.apply(
    lambda x: len(set.intersection(set(x.purchasedProducts.split(',')), 
                                   set(x.recommendForView1.split(','))))/len(set(x.purchasedProducts.split(','))), 
                                                                                                                 axis=1)

In [26]:
testData['recommendForView5'] = testData.viewedProducts.apply(lambda x: sortedView(x, k = 5))

testData['viewPrecision@5'] = testData.apply(
    lambda x: len(set.intersection(set(x.purchasedProducts.split(',')),
                                   set(x.recommendForView5.split(','))))/5, axis=1)

testData['viewRecall@5'] = testData.apply(
    lambda x: len(set.intersection(set(x.purchasedProducts.split(',')), 
                                   set(x.recommendForView5.split(','))))/len(set(x.purchasedProducts.split(','))), 
                                                                                                                 axis=1)

In [27]:
testData['sortedPurchase'] = testData.viewedProducts.apply(lambda x: sortedPurchase(x, k = 0))
testData['recommendForPurchase1'] = testData.viewedProducts.apply(lambda x: sortedPurchase(x, k = 1))


testData['purchasePrecision@1'] = testData.apply(
    lambda x: len(set.intersection(set(x.purchasedProducts.split(',')), 
                                   set(x.recommendForPurchase1.split(',')))), axis=1)
testData['purchaseRecall@1'] = testData.apply(
    lambda x: len(set.intersection(set(x.purchasedProducts.split(',')), 
                                   set(x.recommendForPurchase1.split(','))))/(len(set(x.purchasedProducts.split(',')))), 
                                                                                                                 axis=1)

In [28]:
testData['recommendForPurchase5'] = testData.viewedProducts.apply(lambda x: sortedPurchase(x, k = 5))
testData['purchasePrecision@5'] = testData.apply(
    lambda x: len(set.intersection(set(x.purchasedProducts.split(',')), 
                                   set(x.recommendForPurchase5.split(','))))/5, axis=1)
testData['purchaseRecall@5'] = testData.apply(
    lambda x: len(set.intersection(set(x.purchasedProducts.split(',')), 
                                   set(x.recommendForPurchase5.split(','))))/len(set(x.purchasedProducts.split(','))), 
                                                                                                                 axis=1)

In [29]:
testData.head()

,viewedProducts,purchasedProducts,sortedViews,recommendForView1,viewPrecision@1,viewRecall@1,recommendForView5,viewPrecision@5,viewRecall@5,sortedPurchase,recommendForPurchase1,purchasePrecision@1,purchaseRecall@1,recommendForPurchase5,purchasePrecision@5,purchaseRecall@5
7,"63,68,69,70,66,61,59,61,66,68","66,63","63,68,66,61,59,69,70",63,1,0.5,"63,68,66,61,59",0.4,1.0,"63,68,69,70,66,61,59",63,1,0.5,"63,68,69,70,66",0.4,1.0
14,"158,159,160,159,161,162",162,"158,162,160,159,161",158,0,0.0,"158,162,160,159,161",0.2,1.0,"158,162,160,159,161",158,0,0.0,"158,162,160,159,161",0.2,1.0
19,"200,201,202,203,204","201,205","204,202,203,200,201",204,0,0.0,"204,202,203,200,201",0.2,0.5,"204,202,200,201,203",204,0,0.0,"204,202,200,201,203",0.2,0.5
34,"371,372,371","371,373","371,372",371,1,0.5,"371,372",0.2,0.5,"371,372",371,1,0.5,"371,372",0.2,0.5
40,422,422,422,422,1,1.0,422,0.2,1.0,422,422,1,1.0,422,0.2,1.0


Для данных алгоритмов выпишим AverageRecall@1, AveragePrecision@1, AverageRecall@5, AveragePrecision@5 на обучающей и тестовых выборках.

In [31]:
print(trainData['viewPrecision@1'].mean())
print(trainData['viewRecall@1'].mean())
print(trainData['viewPrecision@5'].mean())
print(trainData['viewRecall@5'].mean())

0.5121951219512195
0.4426343165949593
0.21252771618625918
0.8246918247126122


In [32]:
print(trainData['purchasePrecision@1'].mean())
print(trainData['purchaseRecall@1'].mean())
print(trainData['purchasePrecision@5'].mean())
print(trainData['purchaseRecall@5'].mean())

0.8037694013303769
0.6884494924267653
0.2525498891352649
0.9263073024228787


In [33]:
print(testData['viewPrecision@1'].mean())
print(testData['viewRecall@1'].mean())
print(testData['viewPrecision@5'].mean())
print(testData['viewRecall@5'].mean())

0.48130968622100956
0.41733266203252534
0.2037653478854079
0.8000340663538579


In [34]:
print(testData['purchasePrecision@1'].mean())
print(testData['purchaseRecall@1'].mean())
print(testData['purchasePrecision@5'].mean())
print(testData['purchaseRecall@5'].mean())

0.5276944065484311
0.4606201666660294
0.21009549795362173
0.8201874337490194


Как мы видим, качество рекомендаций на тестовой выборке получилось выше в среднем на 0.2